In [1]:
from neuron import h, units, rxd, load_mechanisms
from neuron.units import mV, ms, um, mM
import math
from matplotlib import pyplot as plt
h.load_file("stdrun.hoc")
%matplotlib inline
from itertools import product
import plotly.express as px
import pandas as pd

ModuleNotFoundError: No module named 'neuron'

In [2]:
my_cell = h.Section(name = "my_cell")

NameError: name 'h' is not defined

# Where

For our model we have a few regions where reactions will occur and species will live:

* Cytosol - `cyt`
* ER - `er`
* Mitochondria - `mito`
* Microdomain - `mdm`
* Extracellular Space - `ecs`

## Cytosol

Here we use a `rxd.Region` to define the cytosol. We use `nrn_region="i"` to specify that this is just on the inside of the membrane. Since we will have multiple regions inside our section, we will use `geometry=rxd.FractionalVolume` to specify what portion of the volume inside the cell is cytosol and `surface_fraction=1` because we will assume all points of the plasma membrane are in contact with the cytosol. The first argument for `geometry=rxd.FractionalVolume` specifies the portion of the cell volume that the region takes up. From a quick google search, the cytosol makes up about 70% of the volume in the cell. This should be investigated further.

In [3]:
cyt = rxd.Region(my_cell,
                 nrn_region="i",
                 geometry=rxd.FractionalVolume.7(0, surface_fraction=1),
                 name="cyt")

SyntaxError: invalid syntax (2038773769.py, line 3)

## Endoplasmic Reticulum

`surface_fraction=0` will be used here because we will assume that the er doesn't contact the plasma membrane. A quick google search showed that slightly more than 10% of cell volume can be made up by ER's.

In [6]:
er = rxd.Region(my_cell,
                geometry=rxd.FractionalVolume(0.10, surface_fraction=0),
                name="er")

## Mitochondria

Quick google search (https://www.ncbi.nlm.nih.gov/books/NBK26894/#:~:text=For%20technical%20reasons%2C%20many%20of,fifth%20of%20the%20cell%20volume.) showed that mitochondria can take up to one-fifth of cell volume. Similarly to ER will will assume `surface_fraction=0`.

In [7]:
mito = rxd.Region(my_cell,
                  geometry=rxd.FractionalVolume(0.20, surface_fraction=0),
                  name="mito")

## Membranes separating cytosol from ER and mitochondria

`rxd.MultiCompartmentReactions` crossing this membrane will be scaled by its surface area. Given that we are assuming that all chunks our cell have ER, mitochondria, and cytosol mixed together in the same way, it follows that the area of the membrane within a volume should be proportional to the size of that volume. We indicate this using the `rxd.DistributedBoundary` geometry. The argument inside the distributed boundary indicates the amount of organelle boundary per cubi